# Music Recommender System using Apache Spark and Python

In [7]:
from pyspark.mllib.recommendation import *
import random
from operator import *

In [8]:
def parser1(s):
    temp = s.split(" ")
    return (int(temp[0]),int(temp[1]),int(temp[2]))
def parser2(s):
    temp = s.split("\t")
    return (int(temp[0]),int(temp[1]))
    return list
def parser3(s):
    temp = s.split("\t")
    return (int(temp[0]),temp[1])
temp = sc.textFile("D:/HKUST/Big Data Computing/Project/Reference/MusicRecommendation-Spark--master/user_artist_data_small.txt").map(parser1)
artistAlias = sc.textFile("D:/HKUST/Big Data Computing/Project/Reference/MusicRecommendation-Spark--master/artist_alias_small.txt").map(parser2)
aliasDic = artistAlias.collectAsMap()
userArtistData = temp.map(lambda x: (x[0], aliasDic.get(x[1], x[1]), x[2]))
artistData = sc.textFile("D:/HKUST/Big Data Computing/Project/Reference/MusicRecommendation-Spark--master/artist_data_small.txt").map(parser3)
#print aliasDic
userArtistData.take(10)

[(1059637, 1000010, 238),
 (1059637, 1000049, 1),
 (1059637, 1000056, 1),
 (1059637, 1000062, 11),
 (1059637, 1000094, 1),
 (1059637, 1000112, 423),
 (1059637, 1000113, 5),
 (1059637, 1000114, 2),
 (1059637, 1000123, 2),
 (1059637, 1000130, 19129)]

In [9]:
artistAlias.take(10)

[(1027859, 1252408),
 (1017615, 668),
 (6745885, 1268522),
 (1018110, 1018110),
 (1014609, 1014609),
 (6713071, 2976),
 (1014175, 1014175),
 (1008798, 1008798),
 (1013851, 1013851),
 (6696814, 1030672)]

In [10]:
artistData.take(10)

[(1240105, 'André Visior'),
 (1240113, 'riow arai'),
 (1240132, 'Outkast & Rage Against the Machine'),
 (6776115, '小松正夫'),
 (1030848, "Raver's Nature"),
 (6671601, 'Erguner, Kudsi'),
 (1106617, 'Bloque'),
 (1240185, 'Lexy & K. Paul'),
 (6671631, 'Rev. W.M. Mosley'),
 (6671632, 'Labelle, Patti')]

In [12]:
def transform(x):
    l = list(x[1])
    counts = []
    for i in range(0,len(l)):
        play = l[i][2]
        counts.append(play)
    total = sum(counts)
    avg = total/len(l)
    return (x[0],total,avg)  
temp = userArtistData.groupBy(lambda x: x[0]).map(lambda x: (x[0],list(x[1]))).map(transform)
#print temp.collect()
top = temp.takeOrdered(3, key=lambda x: -x[1])
for i in range(0,len(top)):
    print("User " + str(top[i][0]) +" has a total play count of "+ str(top[i][1]) +" and a mean play count of "+ str(top[i][2]) +".")

User 1059637 has a total play count of 674412 and a mean play count of 1878.5849582172702.
User 2064012 has a total play count of 548427 and a mean play count of 9455.637931034482.
User 2069337 has a total play count of 393515 and a mean play count of 1519.3629343629343.


In [15]:
trainData,validationData,testData = userArtistData.randomSplit([6,2,2], 13)
print(trainData.take(3))
print(validationData.take(3))
print(testData.take(3))
print(trainData.count())
print(validationData.count())
print(testData.count())
trainData.cache()
validationData.cache()
testData.cache()

[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000114, 2)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000123, 2)]
[(1059637, 1000094, 1), (1059637, 1000112, 423), (1059637, 1000113, 5)]
29552
9907
10022


PythonRDD[58] at RDD at PythonRDD.scala:53

In [20]:
def modelEval(model,validation):
    import operator
    userIds = validation.map(lambda x: x[0]).distinct().collect()
    #print len(userIds)
    artistIds = list(artistData.collectAsMap().keys())
    temp = trainData.map(lambda x: (x[0], x[1])).filter(lambda x: x[0] in userIds).groupByKey().map(lambda x: (x[0], list(x[1])))
    valiSet = temp.flatMap(lambda x: [(x[0],a) for a in artistIds if a not in x[1]])
    # .groupBy(lambda x:x[0]).map(lambda x:(x[0],len(list(elmt[1] for elmt in x[1])))).collect()
    # temp = artistData.flatMap(lambda x: [(y, x[0]) for y in userIds])
    # valiSet = temp.filter(lambda x: x not in trainSet)
    # print valiSet.groupBy(lambda x:x[0]).map(lambda x:(x[0],len(list(elmt[1] for elmt in x[1])))).collect()
    # print 'here'
    predictions = bestModel.predictAll(valiSet).map(lambda x: (x[0], x[1], x[2])).groupBy(lambda x:x[0]).map(lambda x:(x[0],sorted(list(x[1]),key=operator.itemgetter(2),reverse=True)))
    predictions = predictions.map(lambda x:(x[0],list(elmt[1] for elmt in x[1])))
    cmprSet = validation.groupBy(lambda x:x[0]).map(lambda x:(x[0],list(x[1]))).map(lambda x:(x[0],list(elmt[1] for elmt in x[1])))
    p = predictions.collect()
    cdic =  cmprSet.collectAsMap()  
    scores = []
    for i in range(0,len(p)):
        user = p[i][0]
        plist = p[i][1]
        clist = cdic[user]
        plen = len(plist)
        clen = len(clist)
        #print 'ppppppppp'
        #print plen
        #print 'cccccccccc'
        #print clen
        count = 0
        score = 0
        if plen >= clen:
            for j in range(0,clen):
                if plist[j] in clist:
                    count +=1
        score = float(float(count)/clen)
        scores.append(score)
    return sum(scores)/len(scores)

In [21]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
for elmt in range(2,11,2):
    bestModel = ALS.trainImplicit(trainData, rank=elmt)
    print('The model score for rank ' + str(elmt) + ' is '+ str(modelEval(bestModel,validationData)))

The model score for rank 2 is 0.06086241846519619
The model score for rank 4 is 0.07256298165544033
The model score for rank 6 is 0.07832541606495551
The model score for rank 8 is 0.07270293840529879
The model score for rank 10 is 0.07766516111812562


In [22]:
bestModel = ALS.trainImplicit(trainData, rank=6)
print(modelEval(bestModel, testData))

0.07896140645360168


In [24]:
temp = bestModel.recommendProducts(1059637, 5)
lists = []
for i in range(0,len(temp)):
    lists.append(temp[i][1])
    print("Artist "+str(i)+": "+str(artistData.lookup(temp[i][1])[0],))

Artist 0: Something Corporate
Artist 1: My Chemical Romance
Artist 2: The Used
Artist 3: Green Day
Artist 4: U2
